In [4]:
# import modules
import torchio as tio


# load in the nifti file with torchio
image_path = '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii'

image = tio.ScalarImage(image_path)


# estimate the distance between the features of interest

# calculate teh resample ratio from this

# put that in 
# python main.py infer configs/cystisoma_corneas.yaml -v /path/to/volume.nii -m .logs/cystisoma_corneas/lightning_logs/version_13/checkpoints/epoch=2145-step=657024.ckpt -rr YOUR_RESAMPLE_RATIO

In [5]:
image

ScalarImage(shape: (1, 1446, 1657, 1964); spacing: (1.00, 1.00, 1.00); orientation: LPS+; path: "/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii")

In [6]:
# view file with napari
import napari


viewer = napari.view_image(image.numpy(), name='image')


/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-package

In [11]:
import pandas as pd
RR = pd.read_csv('/home/jake/projects/dhr/FEG20190213_01_corneas_resample_ratio.csv')
RR

,index,axis-0,axis-1,axis-2,axis-3
0,0.0,0.0,330.0,448.622806,685.866815
1,1.0,0.0,330.0,473.192624,666.119111
2,2.0,0.0,323.0,438.748954,705.384895
3,3.0,0.0,312.0,463.778021,688.851933
4,4.0,0.0,309.0,492.481079,675.074465


In [14]:
from scipy.spatial import distance_matrix

coordinates = RR[['axis-1', 'axis-2', 'axis-3']].values
dist_matrix = distance_matrix(coordinates, coordinates)


In [15]:
import numpy as np
np.fill_diagonal(dist_matrix, np.inf)

# Find the nearest neighbor distance for each point
nearest_distances = np.min(dist_matrix, axis=1)

# Add the nearest distances to the DataFrame
RR['nearest_distance'] = nearest_distances
RR

,index,axis-0,axis-1,axis-2,axis-3,nearest_distance
0,0.0,0.0,330.0,448.622806,685.866815,22.966245
1,1.0,0.0,330.0,473.192624,666.119111,29.887169
2,2.0,0.0,323.0,438.748954,705.384895,22.966245
3,3.0,0.0,312.0,463.778021,688.851933,23.719011
4,4.0,0.0,309.0,492.481079,675.074465,29.887169


In [17]:
mean_nearest_distance = RR['nearest_distance'].mean()
mean_nearest_distance

25.88516765563356

In [19]:
resample_ratio = mean_nearest_distance/10
resample_ratio

2.588516765563356

In [20]:
10 / mean_nearest_distance

0.38632162375906554

In [23]:
!python main.py infer -h

usage: main.py [-h] [--volume_path VOLUME_PATH]
               [--sql_storage_url SQL_STORAGE_URL]
               [--starting_weights_path STARTING_WEIGHTS_PATH]
               [--model_path MODEL_PATH] [--profile] [--num_steps NUM_STEPS]
               [--check_val_every_n_epoch CHECK_VAL_EVERY_N_EPOCH]
               [--resample_ratio_path RESAMPLE_RATIO_PATH]
               [--bbox_path BBOX_PATH] [--n_x_dirs N_X_DIRS]
               [--n_y_dirs N_Y_DIRS] [--n_z_dirs N_Z_DIRS]
               [--resample_ratio RESAMPLE_RATIO] [--already_resampled]
               [--training_data_histogram] [--infer_with_max_filter_fun]
               [--average_threshold AVERAGE_THRESHOLD]
               [--inference_peak_min_val INFERENCE_PEAK_MIN_VAL]
               {train,tune,infer,locate_peaks} config_path

Train, hyperparameter tune or infer with a deep heatmap regression model

positional arguments:
  {train,tune,infer,locate_peaks}
                        Mode of operation
  config_path      

In [2]:
!python main.py infer configs/cystisoma_corneas.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m .logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 1 -ny 1 -nz 1

Traceback (most recent call last):
  File "/home/jake/projects/dhr/main.py", line 403, in <module>
    main()
  File "/home/jake/projects/dhr/main.py", line 349, in main
    checkpoint = max(list_of_files, key=os.path.getctime)
ValueError: max() arg is an empty sequence


In [2]:
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 1
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 1.5


TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_leng

 43%|██████████████████▍                        | 30/70 [00:46<00:51,  1.29s/it]


 44%|███████████████████                        | 31/70 [00:47<00:54,  1.39s/it]


 46%|███████████████████▋                       | 32/70 [00:49<00:55,  1.47s/it]


 47%|████████████████████▎                      | 33/70 [00:50<00:56,  1.52s/it]


 49%|████████████████████▉                      | 34/70 [00:52<00:55,  1.54s/it]


 50%|█████████████████████▌                     | 35/70 [00:54<00:55,  1.58s/it]


 51%|██████████████████████                     | 36/70 [00:55<00:54,  1.59s/it]


 53%|██████████████████████▋                    | 37/70 [00:57<00:52,  1.59s/it]


 54%|███████████████████████▎                   | 38/70 [00:59<00:50,  1.59s/it]


 56%|███████████████████████▉                   | 39/70 [01:00<00:49,  1.60s/it]


 57%|████████████████████████▌                  | 40/70 [01:02<00:48,  1.62s/it]


 59%|█████████████████████████▏                 | 41/70 [01:03<00:47,  1.63s/it]


 60%

 77%|█████████████████████████████████▏         | 54/70 [01:16<00:25,  1.58s/it]


 79%|█████████████████████████████████▊         | 55/70 [01:17<00:23,  1.60s/it]


 80%|██████████████████████████████████▍        | 56/70 [01:19<00:21,  1.51s/it]


 81%|███████████████████████████████████        | 57/70 [01:20<00:17,  1.32s/it]


 83%|███████████████████████████████████▋       | 58/70 [01:21<00:15,  1.31s/it]


 84%|████████████████████████████████████▏      | 59/70 [01:22<00:15,  1.40s/it]


 86%|████████████████████████████████████▊      | 60/70 [01:24<00:15,  1.51s/it]


 87%|█████████████████████████████████████▍     | 61/70 [01:26<00:13,  1.51s/it]


 89%|██████████████████████████████████████     | 62/70 [01:27<00:12,  1.56s/it]


 90%|██████████████████████████████████████▋    | 63/70 [01:29<00:11,  1.59s/it]


 91%|███████████████████████████████████████▎   | 64/70 [01:31<00:09,  1.61s/it]


 93%|███████████████████████████████████████▉   | 65/70 [01:32<00:08,  1.62s/it]


 94%

  3%|█▎                                          | 2/70 [00:02<01:09,  1.02s/it]


  4%|█▉                                          | 3/70 [00:03<01:13,  1.09s/it]


  6%|██▌                                         | 4/70 [00:04<01:11,  1.08s/it]


  7%|███▏                                        | 5/70 [00:05<01:08,  1.06s/it]


  9%|███▊                                        | 6/70 [00:06<01:08,  1.06s/it]


 10%|████▍                                       | 7/70 [00:07<01:07,  1.07s/it]


 11%|█████                                       | 8/70 [00:08<01:06,  1.08s/it]


 13%|█████▋                                      | 9/70 [00:09<01:05,  1.08s/it]


 14%|██████▏                                    | 10/70 [00:10<01:04,  1.08s/it]


 16%|██████▊                                    | 11/70 [00:11<01:00,  1.03s/it]


 17%|███████▎                                   | 12/70 [00:12<00:56,  1.03it/s]


 19%|███████▉                                   | 13/70 [00:13<01:04,  1.14s/it]


 20%

 40%|█████████████████▏                         | 28/70 [00:38<00:47,  1.14s/it]


 41%|█████████████████▊                         | 29/70 [00:39<00:45,  1.12s/it]


 43%|██████████████████▍                        | 30/70 [00:40<00:44,  1.11s/it]


 44%|███████████████████                        | 31/70 [00:41<00:42,  1.10s/it]


 46%|███████████████████▋                       | 32/70 [00:42<00:41,  1.09s/it]


 47%|████████████████████▎                      | 33/70 [00:43<00:40,  1.09s/it]


 49%|████████████████████▉                      | 34/70 [00:44<00:37,  1.04s/it]


 50%|█████████████████████▌                     | 35/70 [00:45<00:34,  1.02it/s]


 51%|██████████████████████                     | 36/70 [00:46<00:32,  1.05it/s]


 53%|██████████████████████▋                    | 37/70 [00:48<00:37,  1.13s/it]


 54%|███████████████████████▎                   | 38/70 [00:49<00:40,  1.27s/it]


 56%|███████████████████████▉                   | 39/70 [00:51<00:42,  1.37s/it]


 57%

...

 70%|██████████████████████████████             | 49/70 [01:10<00:30,  1.46s/it]


 71%|██████████████████████████████▋            | 50/70 [01:12<00:30,  1.54s/it]


 73%|███████████████████████████████▎           | 51/70 [01:14<00:29,  1.57s/it]


 74%|███████████████████████████████▉           | 52/70 [01:15<00:28,  1.59s/it]


 76%|████████████████████████████████▌          | 53/70 [01:17<00:27,  1.61s/it]


 77%|█████████████████████████████████▏         | 54/70 [01:19<00:26,  1.63s/it]


 79%|█████████████████████████████████▊         | 55/70 [01:20<00:24,  1.65s/it]


 80%|██████████████████████████████████▍        | 56/70 [01:22<00:23,  1.64s/it]


 81%|███████████████████████████████████        | 57/70 [01:24<00:21,  1.65s/it]


 83%|███████████████████████████████████▋       | 58/70 [01:25<00:19,  1.61s/it]


 84%|████████████████████████████████████▏      | 59/70 [01:26<00:16,  1.54s/it]


 86%|████████████████████████████████████▊      | 60/70 [01:28<00:15,  1.58s/it]


 87%

Initialising patch_loader and aggregator...
Starting inference...



  0%|                                                    | 0/70 [00:00<?, ?it/s]


  1%|▋                                           | 1/70 [00:01<01:57,  1.71s/it]


  3%|█▎                                          | 2/70 [00:03<01:52,  1.65s/it]


  4%|█▉                                          | 3/70 [00:04<01:50,  1.65s/it]


  6%|██▌                                         | 4/70 [00:06<01:44,  1.58s/it]


  7%|███▏                                        | 5/70 [00:07<01:35,  1.47s/it]


  9%|███▊                                        | 6/70 [00:09<01:33,  1.46s/it]


 10%|████▍                                       | 7/70 [00:10<01:33,  1.48s/it]


 11%|█████                                       | 8/70 [00:12<01:34,  1.52s/it]


 13%|█████▋                                      | 9/70 [00:13<01:34,  1.55s/it]


 14%|██████▏                                    | 10/70 [00:15<01:34,  1.58s/it]


 16%|██████▊      

 33%|██████████████▏                            | 23/70 [00:28<00:51,  1.10s/it]


 34%|██████████████▋                            | 24/70 [00:29<00:50,  1.09s/it]


 36%|███████████████▎                           | 25/70 [00:30<00:47,  1.05s/it]


 37%|███████████████▉                           | 26/70 [00:31<00:43,  1.02it/s]


 39%|████████████████▌                          | 27/70 [00:32<00:43,  1.01s/it]


 40%|█████████████████▏                         | 28/70 [00:33<00:49,  1.19s/it]


 41%|█████████████████▊                         | 29/70 [00:35<00:49,  1.21s/it]


 43%|██████████████████▍                        | 30/70 [00:36<00:50,  1.25s/it]


 44%|███████████████████                        | 31/70 [00:38<00:53,  1.37s/it]


 46%|███████████████████▋                       | 32/70 [00:39<00:55,  1.45s/it]


 47%|████████████████████▎                      | 33/70 [00:41<00:55,  1.50s/it]


 49%|████████████████████▉                      | 34/70 [00:43<00:56,  1.57s/it]


 50%

 66%|████████████████████████████▎              | 46/70 [01:03<00:25,  1.08s/it]


 67%|████████████████████████████▊              | 47/70 [01:05<00:28,  1.24s/it]


 69%|█████████████████████████████▍             | 48/70 [01:06<00:29,  1.35s/it]


 70%|██████████████████████████████             | 49/70 [01:08<00:28,  1.37s/it]


 71%|██████████████████████████████▋            | 50/70 [01:09<00:28,  1.45s/it]


 73%|███████████████████████████████▎           | 51/70 [01:11<00:25,  1.36s/it]


 74%|███████████████████████████████▉           | 52/70 [01:12<00:25,  1.42s/it]


 76%|████████████████████████████████▌          | 53/70 [01:14<00:25,  1.51s/it]


 77%|█████████████████████████████████▏         | 54/70 [01:15<00:24,  1.54s/it]


 79%|█████████████████████████████████▊         | 55/70 [01:17<00:23,  1.57s/it]


 80%|██████████████████████████████████▍        | 56/70 [01:19<00:22,  1.58s/it]


 81%|███████████████████████████████████        | 57/70 [01:20<00:20,  1.60s/it]


 83%

Saving prediction to ./output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_1.5_prediction.nii
Locating peaks...
Saving peaks in resampled space...
saving to output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_1.5_prediction_peak_min_val_0_2_method_center_of_mass.resampled_space_peaks.csv
Converting peaks to original image space...
Resample ratio: 2.588516765563356
running inference with bbox if supplied
Saving peaks in original image space...
saving to output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_1.5_prediction_peak_min_val_0_2_method_center_of_mass.peaks.csv
[[1167.42106127 1307.20096661  924.10048531]
 [1185.54067863 1182.95216186  755.84689554]
 [1185.54067863 1234.72249717  846.44498234]
 [1195.89474569 1312.37800014 1066.46890741]
 [1197.18900407 1316.26077529 1006.9330218 ]
 

In [3]:
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 2
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.5

TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_leng

Loading volume...
Resampling volume by a ratio of 2.588516765563356
Image old spacing (1.0, 1.0, 1.0)
Image new spacing (2.588516765563356, 2.588516765563356, 2.588516765563356)
  0%|                                                     | 0/3 [00:00<?, ?it/s]

  0%|                                                     | 0/3 [00:00<?, ?it/s]
Running inference with 0° rotation along the x-axis (1/3),
0° rotation along the y-axis (1/3)
and 0° rotation along the z-axis (1/3)
Creating sampler and applying transform to image...
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torchio/data/subject.py:202: RuntimeWarning: Skipping ToCanonical as it is not invertible
  inverse_transform = history_transform.inverse(warn=warn)
Initialising patch_loader and aggregator...
Starting inference...



  0%|                                                    | 0/70 [00:00<?, ?it/s]


                                                                                
Running inference in 27 direction/

In [1]:
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.5 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.5 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.01 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.02
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.03
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.04 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.05
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.1
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.2 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.3 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.4
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.5 --infer_with_max_filter_fun


#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.02 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.02 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.02 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.03 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.03 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.03 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.02 
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.02 
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.02 
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.03 
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.03 
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.03 

TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_leng

/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_length': 9, 'batch_size': 2, "
 "'config_stem': 'cystisoma_corneas', 'correct_prediction_distance': 6, "
 "'debug_plots': False, 'double_channels_with_depth': True, 'dropout': "
 "0.0175756794552526, 'h

("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_length': 9, 'batch_size': 2, "
 "'config_stem': 'cystisoma_corneas', 'correct_prediction_distance': 6, "
 "'debug_plots': False, 'double_channels_with_depth': True, 'dropout': "
 "0.0175756794552526, 'heatmap_max_length': 64, 'histogram_landmarks_path': "
 "'./landmarks_files/paraphronima_corneas_landmarks.npy', "
 "'histogram_standardisation': False, 'ignore_empty_volumes': True, "
 "'image_suffix': 'image', 'label_suffix': 'corneas', 'learn_sigma': False, "
 "'loss_in_sigma_space': False, 'lr': 0.0424937961824346, 'max_length': 512, "
 "'momentum': 0.510999549987835, 'mse_with_f1': False, 'nesterov': False, "
 "'num_encoding_blocks': 5, 'num_steps': 700000, 'num_workers': 6, "
 "'optimiser': 'SGD', 'out_channels_first_layer': 32, 'output_activation': "
 "'None', 'patch_size': 64, 'peak_min_val': 0.6, 'pin_memory': False, "
 "'pooling_type': 'max', 'random_affine_prob': 0

Loading volume...
Resampling volume by a ratio of 2.588516765563356
Image old spacing (1.0, 1.0, 1.0)
Image new spacing (2.588516765563356, 2.588516765563356, 2.588516765563356)
Loading previously calculated prediction file at ./output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.02_prediction.nii. Delete the file to avoid this behaviour.
Locating peaks...
Saving peaks in resampled space...
saving to output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.02_prediction_peak_min_val_0_1_method_center_of_mass.resampled_space_peaks.csv
Converting peaks to original image space...
Resample ratio: 2.588516765563356
running inference with bbox if supplied
Saving peaks in original image space...
saving to output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.02_prediction_peak_min_val_0_1_method_

Image new spacing (2.588516765563356, 2.588516765563356, 2.588516765563356)
Loading previously calculated prediction file at ./output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.03_prediction.nii. Delete the file to avoid this behaviour.
Locating peaks...
Saving peaks in resampled space...
saving to output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.03_prediction_peak_min_val_0_15_method_center_of_mass.resampled_space_peaks.csv
Converting peaks to original image space...
Resample ratio: 2.588516765563356
running inference with bbox if supplied
Saving peaks in original image space...
saving to output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.03_prediction_peak_min_val_0_15_method_center_of_mass.peaks.csv
[[  82.8325365   632.89234918 1382.26795281]
 [ 105.10310867  620.33454487 

In [2]:
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.01 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.01 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.01 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.01 
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.01 
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.01
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.02 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.02 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.02 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.02 
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.02 
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.02

TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_leng

("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_length': 9, 'batch_size': 2, "
 "'config_stem': 'cystisoma_corneas', 'correct_prediction_distance': 6, "
 "'debug_plots': False, 'double_channels_with_depth': True, 'dropout': "
 "0.0175756794552526, 'heatmap_max_length': 64, 'histogram_landmarks_path': "
 "'./landmarks_files/paraphronima_corneas_landmarks.npy', "
 "'histogram_standardisation': False, 'ignore_empty_volumes': True, "
 "'image_suffix': 'image', 'label_suffix': 'corneas', 'learn_sigma': False, "
 "'loss_in_sigma_space': False, 'lr': 0.0424937961824346, 'max_length': 512, "
 "'momentum': 0.510999549987835, 'mse_with_f1': False, 'nesterov': False, "
 "'num_encoding_blocks': 5, 'num_steps': 700000, 'num_workers': 6, "
 "'optimiser': 'SGD', 'out_channels_first_layer': 32, 'output_activation': "
 "'None', 'patch_size': 64, 'peak_min_val': 0.6, 'pin_memory': False, "
 "'pooling_type': 'max', 'random_affine_prob': 0

Loading volume...
Resampling volume by a ratio of 2.588516765563356
Image old spacing (1.0, 1.0, 1.0)
Image new spacing (2.588516765563356, 2.588516765563356, 2.588516765563356)
Loading previously calculated prediction file at ./output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.01_prediction.nii. Delete the file to avoid this behaviour.
Locating peaks...
Saving peaks in resampled space...
saving to output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.01_prediction_peak_min_val_0_15_method_center_of_mass.resampled_space_peaks.csv
Converting peaks to original image space...
Resample ratio: 2.588516765563356
running inference with bbox if supplied
Saving peaks in original image space...
saving to output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.01_prediction_peak_min_val_0_15_metho

Image new spacing (2.588516765563356, 2.588516765563356, 2.588516765563356)
Loading previously calculated prediction file at ./output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.02_prediction.nii. Delete the file to avoid this behaviour.
Locating peaks...
Saving peaks in resampled space...
saving to output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.02_prediction_peak_min_val_0_5_method_max_filter.resampled_space_peaks.csv
Converting peaks to original image space...
Resample ratio: 2.588516765563356
running inference with bbox if supplied
Saving peaks in original image space...
saving to output/FEG20190213_010.logs_cystisoma_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.02_prediction_peak_min_val_0_5_method_max_filter.peaks.csv
[[1076.82297447 1382.26795281 1110.47369243]
 [1094.94259183 1366.73685222 1092.35407

TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_leng

/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_length': 9, 'batch_size': 2, "
 "'config_stem': 'cystisoma_corneas', 'correct_prediction_distance': 6, "
 "'debug_plots': False, 'double_channels_with_depth': True, 'dropout': "
 "0.0175756794552526, 'h

TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_leng

In [4]:
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.4 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.3 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.2 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.1 --infer_with_max_filter_fun
!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.05 --infer_with_max_filter_fun


TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_leng

TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_leng

TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_corneas/whole/train_images_10/ and ./dataset/cystisoma_corneas/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_leng

In [4]:
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 1 -ny 1 -nz 1 -ipmv 0.2 -at 0.2
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 1 -ny 1 -nz 1 -ipmv 0.3 -at 0.3
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 2 -ny 2 -nz 2 -ipmv 0.5 -at 0.5
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.4
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.4
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.4
!python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.45 -at 0.4



TorchIO version: 0.18.88
Running setup!
Found 1 labelled images with labels in ./dataset/cystisoma_rhabdoms/whole/train_images_10/ and ./dataset/cystisoma_rhabdoms/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/venv/lib/python3.10/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 1 labelled images with labels in ./dataset/cystisoma_rhabdoms/whole/train_images_10/ and ./dataset/cystisoma_rhabdoms/whole/train_labels_10/ for analysis
Training:   1
Validation:  1
Test:       1
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_

In [5]:
import napari
import numpy as np
import torchio as tio


image_path = '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii'
image = tio.ScalarImage(image_path)

# heatmap_path = '/media/jake/Removable backup/Jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_17_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction.nii'
heatmap_path = '/home/jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_19_checkpoints_last_x_3_y_3_z_3_average_threshold_0.4_prediction.nii'

#Make sure to use resampled space csv here.
# y_hat_path = '/media/jake/Removable backup/Jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_17_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction_peak_min_val_0_5_method_center_of_mass.resampled_space_peaks.csv'
y_hat_path = '/home/jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_19_checkpoints_last_x_3_y_3_z_3_average_threshold_0.4_prediction_peak_min_val_0_45_method_center_of_mass.resampled_space_peaks.csv'

heatmap = tio.ScalarImage(heatmap_path)


viewer = napari.view_image(heatmap.numpy(), name='heatmap', contrast_limits=[0, 1])
# viewer.add _image(image.numpy(), name='image') #If you want to view your scan also (not necessary)

# load the coordinates to analyse
def load_coordinates(path, flip_axes=False, mct_path=None):
    locations = np.loadtxt(
        path,
        delimiter=',',
        ndmin=2,
        dtype=np.float64
    )
    if not flip_axes: 
        return locations.tolist()

    if not mct_path:
        raise Exception('You must specify `mct_path` if you need to flip_axes')

    mct = tio.ScalarImage(mct_path)
    locations[:,0] = mct.shape[1] - locations[:,0]
    locations[:,1] = mct.shape[2] - locations[:,1]

    return locations.tolist()


y_hat = load_coordinates(y_hat_path)


viewer.add_points(y_hat)

<Points layer 'Points' at 0x7f0cabe71580>

In [10]:
import h5py
import pandas as pd

# Open the .mat file
with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat', 'r') as file:
    # Access the marked dataset and read all rows
    marked_data = file['save_dat/data/marked'][:]  # Load all rows from the marked dataset

# Convert to a DataFrame for easier manipulation
marked_df = pd.DataFrame(marked_data)

# Print the DataFrame to verify the data
print(marked_df[:10])
print(marked_df[:-10])


         0           1           2           3           4           5     \
0  413.799234  392.224242  418.690916  442.773824  543.599670  549.531006   
1  344.812257  184.796941  205.359985  260.519539   82.198235   76.462900   
2  364.749693  519.553709  486.690526  423.877994  726.151611  758.742294   
3    2.000000    7.000000    6.000000    4.000000  107.000000  104.000000   
4    6.000000   11.000000   11.000000   10.000000  111.000000  112.000000   
5    2.000000    2.000000    2.000000    2.000000    0.000000    0.000000   
6    8.000000   20.000000   21.000000   24.000000   25.000000   26.000000   
7    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   

         6           7           8           9     ...         1632  \
0  522.830477  515.256420  369.343628  334.354801  ...   566.211948   
1   65.022158   67.909723  115.024315  135.410461  ...   682.205432   
2  782.491818  754.416333  638.070637  625.128479  ...   535.117258   
3  105.000000  108.000

In [11]:
import h5py
import numpy as np

# Open the .mat file
with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat', 'r') as file:
    # Load the marked dataset
    marked_data = file['save_dat/data/marked'][:]  # Load all rows

# Transpose the data
transposed_data = marked_data.T  # Transpose the array

# Save to CSV
np.savetxt('marked_data_transposed.csv', transposed_data, delimiter=',', header=",".join([f"Col{i}" for i in range(transposed_data.shape[0])]), comments='', fmt='%f')


In [29]:
import h5py
import pandas as pd

# Open the .mat file and access the marked dataset
with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat', 'r') as file:
    marked_data = file['save_dat/data/marked'][:]  # Load the entire marked dataset

# Convert to a DataFrame for easier manipulation
marked_df = pd.DataFrame(marked_data)

# Print the values in the sixth row to understand what we are filtering
print("Values in the sixth row:", marked_df.iloc[5])

# Identify indices of columns where the sixth row has values 0, 1, 2, or 3
columns_to_keep = marked_df.iloc[5].isin([0, 2, 4, 6])

# Get the column indices for those values
column_indices = marked_df.columns[columns_to_keep]

# Extract the first three rows of the marked data corresponding to the selected columns
filtered_df = marked_df.iloc[:3, column_indices]

# Transpose the filtered DataFrame
transposed_df = filtered_df.transpose()

save_directory = '/home/jake/projects/dhr/dataset/primno_corneas/whole/train_labels_10'  # Change this to your desired path
csv_name = 'Primno_abyssalis_FEG221018_356_labels_corneas.csv'

# Combine directory and filename to create the full path
csv_path = os.path.join(save_directory, csv_name)

# Save to CSV using the specified path
transposed_df.to_csv(csv_path, header=False, index=False)

print(f"CSV saved successfully to: {csv_path}"


SyntaxError: unexpected EOF while parsing (835161552.py, line 35)

In [30]:
import h5py
import pandas as pd
import os

# Open the .mat file and access the marked dataset
with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat', 'r') as file:
    marked_data = file['save_dat/data/marked'][:]  # Load the entire marked dataset

# Convert to a DataFrame for easier manipulation
marked_df = pd.DataFrame(marked_data)

# Print the values in the sixth row to understand what we are filtering
print("Values in the sixth row:", marked_df.iloc[5])

# Identify indices of columns where the sixth row has values 0, 1, 2, or 3
columns_to_keep = marked_df.iloc[5].isin([0, 2, 4, 6])

# Get the column indices for those values
column_indices = marked_df.columns[columns_to_keep]

# Extract the first three rows of the marked data corresponding to the selected columns
filtered_df = marked_df.iloc[:3, column_indices]

# Transpose the filtered DataFrame
transposed_df = filtered_df.transpose()

# Specify the directory where you want to save the CSV
save_directory = '/home/jake/projects/dhr/dataset/primno_corneas/whole/train_labels_10'  # Change this to your desired path
csv_name = 'Primno_abyssalis_FEG221018_356_labels_corneas.csv'

# Combine directory and filename to create the full path
csv_path = os.path.join(save_directory, csv_name)

# Ensure the directory exists
os.makedirs(save_directory, exist_ok=True)

# Save to CSV using the specified path
transposed_df.to_csv(csv_path, header=False, index=False)

print(f"CSV saved successfully to: {csv_path}")


Values in the sixth row: 0       2.0
1       2.0
2       2.0
3       2.0
4       0.0
       ... 
1637    5.0
1638    5.0
1639    1.0
1640    1.0
1641    1.0
Name: 5, Length: 1642, dtype: float64
CSV saved successfully to: /home/jake/projects/dhr/dataset/primno_corneas/whole/train_labels_10/Primno_abyssalis_FEG221018_356_labels_corneas.csv


In [25]:
import h5py
import pandas as pd

# Open the .mat file and access the marked dataset
with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat', 'r') as file:
    marked_data = file['save_dat/data/marked'][:]  # Load the entire marked dataset

# Convert to a DataFrame for easier manipulation
marked_df = pd.DataFrame(marked_data)

# Print the values in the sixth row
row_6_values = marked_df.iloc[5]
print(row_6_values)


0       2.0
1       2.0
2       2.0
3       2.0
4       0.0
       ... 
1637    5.0
1638    5.0
1639    1.0
1640    1.0
1641    1.0
Name: 5, Length: 1642, dtype: float64


In [31]:
# To convert mat for Primno_abyssalis_FEG221018_356 into rhabdoms raw labels csv
import h5py
import pandas as pd
import os

with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat', 'r') as file:
    marked_data = file['save_dat/data/marked'][:]  # Load the entire marked dataset

marked_df = pd.DataFrame(marked_data)


columns_to_keep = marked_df.iloc[5].isin([1, 3, 5, 7]) # correct indices for rhabdoms
column_indices = marked_df.columns[columns_to_keep]
filtered_df = marked_df.iloc[:3, column_indices]
transposed_df = filtered_df.transpose()

# Specify the directory where you want to save the CSV
save_directory = '/home/jake/projects/dhr/dataset/raw_labels'  
csv_name = 'Primno_abyssalis_FEG221018_356_labels_rhaddoms.csv'

csv_path = os.path.join(save_directory, csv_name)

os.makedirs(save_directory, exist_ok=True)

transposed_df.to_csv(csv_path, header=False, index=False)

In [32]:
!python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_3/ -rr 3.290426824776052 -nx 1 -ny 1 -nz 1 -ipmv 0.5 -at 0.1


TorchIO version: 0.18.88
Running setup!
Found 2 labelled images with labels in ./dataset/primno_corneas/whole/train_images_10/ and ./dataset/primno_corneas/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 2 labelled images with labels in ./dataset/primno_corneas/whole/train_images_10/ and ./dataset/primno_corneas/whole/train_labels_10/ for analysis
Training:   2
Validation:  2
Test:       2
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_length': 9, 'batch_size': 2, "
 "'config_stem': 'primno_corneas', 'correct_prediction_distance': 6, "
 "'debug_plots': False, 'double_channels_with_depth': True, 'dropout': "
 "0.0175756794552526, 'heatmap_max_length': 64, 'histogram_landmarks_pat

In [29]:
import napari
import numpy as np
import torchio as tio


image_path = '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii'
image = tio.ScalarImage(image_path)

# heatmap_path = '/media/jake/Removable backup/Jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_17_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction.nii'
heatmap_path = '/home/jake/projects/dhr/output/Primno_abyssalis_FEG221101_359.logs_primno_rhabdoms_lightning_logs_version_1_checkpoints_last_x_1_y_1_z_1_average_threshold_0.4_prediction.nii'

#Make sure to use resampled space csv here.
# y_hat_path = '/media/jake/Removable backup/Jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_17_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction_peak_min_val_0_5_method_center_of_mass.resampled_space_peaks.csv'
y_hat_path = '/home/jake/projects/dhr/output/Primno_abyssalis_FEG221101_359.logs_primno_rhabdoms_lightning_logs_version_1_checkpoints_last_x_1_y_1_z_1_average_threshold_0.4_prediction_peak_min_val_0_4_method_center_of_mass.resampled_space_peaks.csv'

heatmap = tio.ScalarImage(heatmap_path)


viewer = napari.view_image(heatmap.numpy(), name='heatmap', contrast_limits=[0, 1])
# viewer.add_image(image.numpy(), name='image') #If you want to view your scan also (not necessary)

# load the coordinates to analyse
def load_coordinates(path, flip_axes=False, mct_path=None):
    locations = np.loadtxt(
        path,
        delimiter=',',
        ndmin=2,
        dtype=np.float64
    )
    if not flip_axes: 
        return locations.tolist()

    if not mct_path:
        raise Exception('You must specify `mct_path` if you need to flip_axes')

    mct = tio.ScalarImage(mct_path)
    locations[:,0] = mct.shape[1] - locations[:,0]
    locations[:,1] = mct.shape[2] - locations[:,1]

    return locations.tolist()


y_hat = load_coordinates(y_hat_path)


viewer.add_points(y_hat)

<Points layer 'Points' at 0x7f81a36a74c0>

In [6]:
pip install light-the-torch && ltt install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.8 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.5/857.5 MB 4.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Obtaining dependency information for nvidia-cuda-nvrtc-cu11==11.8.89 from https://files.pythonhosted.org/packages/60/44/202e027c224c26e15a53f01c5c7604c7f6b4fd368882d3164ea08fead207/nvidia_cuda_nvrtc_cu11-11.8.89-py3-none-manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for nvidia-cuda-runtime-cu11==11.8.89 from https://files.pythonhosted.org/packages/a6/ec/a540f28b31de7bc1ed49eecc72035d4cb77db88ead1d42f7bfa5ae407ac6/nvidia_cuda_runtime_cu11-11.8.89-py3-none-manylinux2014_x86_64.whl.metadata
  Obtaining dependency information fo

In [8]:
pip install torchio

  Obtaining dependency information for torchio from https://files.pythonhosted.org/packages/74/2d/88c7734d4f905b058df1673f18b7660b14a3dede2228c84d9f7469e9785a/torchio-0.20.0-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 691.3 kB/s eta 0:00:00MB/s eta 0:00:01
  Obtaining dependency information for Deprecated from https://files.pythonhosted.org/packages/20/8d/778b7d51b981a96554f29136cd59ca7880bf58094338085bcf2a979a0e6a/Deprecated-1.2.14-py2.py3-none-any.whl.metadata
  Obtaining dependency information for SimpleITK!=2.0.*,!=2.1.1.1 from https://files.pythonhosted.org/packages/48/f8/3f00cc6d4f11b3cd934e3024c5be71ffc6d30d4620a16de7d194381f92f9/SimpleITK-2.4.0-cp311-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for humanize from https://files.pythonhosted.org/packages/8f/49/a29c79bea335e52fb512a43faf84998c184c87fef82c65f568f8c56f2642/humanize-4.10.0-py3-none-any.whl.metadata
  Obtaining dependency inf

In [11]:
pip install "napari[all]"

  Obtaining dependency information for napari[all] from https://files.pythonhosted.org/packages/7e/c8/6bcaf27758397c8f1da906113bc99a9a4bbbc062835c2bff4fb2c3406753/napari-0.5.4-py3-none-any.whl.metadata
  Obtaining dependency information for app-model<0.4.0,>=0.3.0 from https://files.pythonhosted.org/packages/d7/2d/dd2f88552a243bf7b689b15bd8a099ffd5dbaca55442c5b560a125ad7d39/app_model-0.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachey>=0.2.1 from https://files.pythonhosted.org/packages/57/f0/e24f3e5d5d539abeb783087b87c26cfb99c259f1126700569e000243745a/cachey-0.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for magicgui>=0.7.0 from https://files.pythonhosted.org/packages/b8/06/778d6460b2fb980834bbd32a39f4054ec1e85fefcaefb913ae5c9b58a35e/magicgui-0.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for napari-console>=0.0.9 from https://files.pythonhosted.org/packages/20/df/c7287cadaa92474b20aa7a59d999e49978ee9db416eff636f588e58a1

In [1]:

import torchio as tio

image_path = '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii'
image = tio.ScalarImage(image_path)



In [15]:
napari

<module 'napari' from '/home/jake/anaconda3/lib/python3.12/site-packages/napari/__init__.py'>

In [2]:
# view file with napari
import napari


viewer = napari.view_image(image.numpy(), name='image')

/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-package

/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-package

/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-packages/napari/layers/base/base.py:1540: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/home/jake/projects/dhr/venv/lib/python3.9/site-package

In [3]:
import pandas as pd

RR = pd.read_csv('/home/jake/projects/dhr/Primno_abyssalis_FEG221101_359_rhabdoms_estimating_resampling_ratio.csv')

RR

,index,axis-0,axis-1,axis-2
0,0.0,869.0,482.727050,109.797861
1,1.0,869.0,516.710116,127.396234
2,2.0,869.0,545.838457,146.208288
3,3.0,869.0,573.146278,171.695587
4,4.0,869.0,591.351491,192.328162
5,5.0,869.0,613.197747,217.208621
6,6.0,869.0,635.044004,240.268558
7,7.0,869.0,641.112408,260.901133
8,8.0,898.0,537.342691,142.567245
9,9.0,898.0,565.257352,155.917735


In [4]:
from scipy.spatial import distance_matrix

coordinates = RR[['axis-0', 'axis-1', 'axis-2']].values
dist_matrix = distance_matrix(coordinates, coordinates) #Create a distance matrix between points
import numpy as np
np.fill_diagonal(dist_matrix, np.inf)

# Find the nearest neighbor distance for each point
nearest_distances = np.min(dist_matrix, axis=1)

# Add the nearest distances to the dataframe
RR['nearest_distance'] = nearest_distances
RR
mean_nearest_distance = RR['nearest_distance'].mean()
mean_nearest_distance
resample_ratio = mean_nearest_distance/10
resample_ratio #Print your RR; use this to make your inference 

2.9493045478545903

In [28]:
# !python main.py infer configs/primno_rhabdoms.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_rhabdoms/lightning_logs/version_1/ -rr 2.9493045478545903 -nx 1 -ny 1 -nz 1 -ipmv 0.5 -at 0.5 --infer_with_max_filter_fun
# !python main.py infer configs/primno_rhabdoms.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_rhabdoms/lightning_logs/version_1/ -rr 2.9493045478545903 -nx 1 -ny 1 -nz 1 -ipmv 0.1 -at 0.1 --infer_with_max_filter_fun
# !python main.py infer configs/primno_rhabdoms.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_rhabdoms/lightning_logs/version_1/ -rr 2.9493045478545903 -nx 1 -ny 1 -nz 1 -ipmv 0.2 -at 0.2 --infer_with_max_filter_fun
# !python main.py infer configs/primno_rhabdoms.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_rhabdoms/lightning_logs/version_1/ -rr 2.9493045478545903 -nx 1 -ny 1 -nz 1 -ipmv 0.1 -at 0.1 # --infer_with_max_filter_fun
!python main.py infer configs/primno_rhabdoms.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_rhabdoms/lightning_logs/version_1/ -rr 2.9493045478545903 -nx 1 -ny 1 -nz 1 -ipmv 0.4 -at 0.4 #--infer_with_max_filter_fun

TorchIO version: 0.18.88
Running setup!
Found 2 labelled images with labels in ./dataset/primno_rhabdoms/whole/train_images_10/ and ./dataset/primno_rhabdoms/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 2 labelled images with labels in ./dataset/primno_rhabdoms/whole/train_images_10/ and ./dataset/primno_rhabdoms/whole/train_labels_10/ for analysis
Training:   2
Validation:  2
Test:       2
("Initiating model using the following config: {'act': 'ReLU', "
 "'balanced_sampler': True, 'balanced_sampler_max_length': 9, 'batch_size': 2, "
 "'config_stem': 'primno_rhabdoms', 'correct_prediction_distance': 6, "
 "'debug_plots': False, 'double_channels_with_depth': True, 'dropout': "
 "0.0175756794552526, 'heatmap_max_length': 64, 'histogram_landmark

In [23]:
!python check_data.py ./configs/primno_rhabdoms.yaml

Running setup!
Found 2 labelled images with labels in ./dataset/primno_rhabdoms/whole/train_images_10/ and ./dataset/primno_rhabdoms/whole/train_labels_10/ for analysis
/home/jake/projects/dhr/deep_radiologist/lightning_modules.py:383: UserWarning: Length of validation subjects is 0. Setting the validation subjects to the same subjects as those used in training. Saved metrics/logs may be misleading.
  warnings.warn(
Found 2 labelled images with labels in ./dataset/primno_rhabdoms/whole/train_images_10/ and ./dataset/primno_rhabdoms/whole/train_labels_10/ for analysis
("Initiating model using the following config: {'seed': 44, 'lr': "
 "0.0424937961824346, 'num_steps': 1100000, 'weight_decay': "
 "1.52709371682483e-06, 'momentum': 0.510999549987835, 'nesterov': False, "
 "'act': 'ReLU', 'output_activation': 'None', 'softargmax': False, "
 "'num_encoding_blocks': 5, 'out_channels_first_layer': 32, "
 "'double_channels_with_depth': True, 'pooling_type': 'max', "
 "'upsampling_type': 'conv